<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span><ul class="toc-item"><li><span><a href="#Logging" data-toc-modified-id="Logging-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Logging</a></span></li><li><span><a href="#Generic-Script-code" data-toc-modified-id="Generic-Script-code-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generic Script code</a></span></li><li><span><a href="#Get-soup-object-for-problem-URL" data-toc-modified-id="Get-soup-object-for-problem-URL-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Get soup object for problem URL</a></span></li><li><span><a href="#Problem-Description:" data-toc-modified-id="Problem-Description:-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Problem Description:</a></span></li></ul></li><li><span><a href="#Problem-Code" data-toc-modified-id="Problem-Code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Problem Code</a></span></li><li><span><a href="#Investigation" data-toc-modified-id="Investigation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Investigation</a></span><ul class="toc-item"><li><span><a href="#How-code-is-laid-out-in-main:" data-toc-modified-id="How-code-is-laid-out-in-main:-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>How code is laid out in main:</a></span></li></ul></li><li><span><a href="#Solution" data-toc-modified-id="Solution-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Solution</a></span></li></ul></div>

### Imports

In [2]:
import logging
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

#### Logging

In [3]:
logging.basicConfig(level=logging.DEBUG)

#### Generic Script code

In [4]:
def _get_response(url):
    response = requests.get(url)
    return response

In [5]:
def get_soup_object(url):
    response = _get_response(url)
    #Insert logic to check
    #if response = 200
    soup = BeautifulSoup(response.text, "html.parser")
    #else handle exceptions
    return soup

In [6]:
def _read_table(table_number, soup):
    table = soup.find_all('table')[table_number]
    rows = table.find_all('tr')
    row_list = list()
    for tr in rows:
        td = tr.find_all('td')
        row = [i.text for i in td]
        row_list.append(row)
    flat_list = [item for sublist in row_list for item in sublist]
    return flat_list

In [7]:
def check_pattern_existence(pattern, string):
    if re.search(pattern, string):
        next_line_indicator = True
    else:
        next_line_indicator = False
    
    return next_line_indicator

In [8]:
def read_table_one(soup, table_number):
    
    flat_list = _read_table(table_number, soup)
    
    _list = []
    dict_keys = []
    dict_values = []
    
    flat_list = flat_list[:13] #Specific - change them ???????????
    
    for i in range(len(flat_list)):
        _list_val = flat_list[i]
        _sublist = _list_val.split(': ') #Specific to this context
        _list.append(_sublist)

    for i in range(len(_list)):
        dict_keys.append(_list[i][0])
        dict_values.append(_list[i][1])
    
    _dict = dict(zip(dict_keys, dict_values))
    
    df_table_1 = pd.DataFrame().append(_dict, ignore_index=True)
    
    return df_table_1

In [9]:
def read_table_two(soup, table_number):
    
    flat_list = _read_table(table_number, soup)
    
    _list = []
    dict_keys = []
    dict_values = []
    
    logging.debug(f"Length of flat_list is: {len(flat_list)}.")
    logging.debug(f"flat_list is: {flat_list}.")
    
    next_line_pattern = '\\n[\\t]+'
    max_range_pattern = ' Max Range: '
    
    for i in range(len(flat_list)):
        _list_val = flat_list[i]
        if check_pattern_existence(max_range_pattern, _list_val) and \
        check_pattern_existence(next_line_pattern, _list_val):
            _sublist_2 = _list_val.split(' Max Range: ')
            _sublist = re.split('\\n[\\t]+', _sublist_2[0])
            _sublist.append(_sublist_2[1]) #Specific to this context
            _list.append(_sublist)
        else:
            _list.append([_list_val])
    
    for i in range(len(_list)):
        dict_keys_template = [f'Sensor {i+1} Name', f'Sensor {i+1} Type', f'Sensor {i+1} Max Range']
        for j in range(len(_list[i])):
            dict_keys.append(dict_keys_template[j])
            dict_values.append(_list[i][j])
    
    _dict = dict(zip(dict_keys, dict_values))
    
    df_table_2 = pd.DataFrame().append(_dict, ignore_index=True)
    
    return df_table_2

In [10]:
def _get_table_header(table, index_list, header_list):
    
    for i in range(len(table)):
        header = table[i].find_all('th')
        header_string = [i.text for i in header]
        index_list.append(i)
        header_list.append(header_string)
        logging.debug(f"Index: {i}")
        logging.debug(f"Header: {header_string}")
    
    return index_list, header_list

In [11]:
def count_tables_on_page(soup):
    table = soup.find_all('table')
    number_of_tables = len(table)
    logging.debug(f"There are {number_of_tables} tables on this page.")
    index_list = []
    header_list = []
    if len(table) > 0:
        index_list, header_list = _get_table_header(table, index_list, header_list)
    else:
        logging.error(f"No tables found.")
    
    table_header_dict = {"index": index_list,
            "header": header_list}
    
    return number_of_tables, table_header_dict

#### Get soup object for problem URL

In [12]:
debug_url = 'http://cmano-db.com/aircraft/4354/'#1526/'3040

In [13]:
soup = get_soup_object(debug_url)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): cmano-db.com:80
DEBUG:urllib3.connectionpool:http://cmano-db.com:80 "GET /aircraft/4354/ HTTP/1.1" 200 None


In [14]:
number_of_tables, table_header_dict = count_tables_on_page(soup)

DEBUG:root:There are 3 tables on this page.
DEBUG:root:Index: 0
DEBUG:root:Header: ['General data:', '']
DEBUG:root:Index: 1
DEBUG:root:Header: ['Sensors / EW:']
DEBUG:root:Index: 2
DEBUG:root:Header: ['Weapons / Loadouts:']


#### Problem Description:
Need to put in a fix to check if table number 1, 2, 3 even exists?
If it doesn't exist, I can not apply any processing to it.

Issue was picked up with http://cmano-db.com/aircraft/1526/

### Problem Code

In [17]:
table = soup.find_all('table')

In [21]:
table[2]

<table class="table table-striped table-hover">
<thead>
<tr>
<th><b><u>Weapons / Loadouts:</u></b></th>
</tr>
</thead>
<tbody>
<tr><td><b><a href="weapon/2650/">330 liter Drop Tank</a></b> -
				 Drop Tank<br/>
</td></tr><tr><td> <a class="preview pull-right" href="images/DB3000/weapon_1906.jpg" target="_blank"><i class="fa fa-picture-o"></i></a><b><a href="weapon/1906/">GBU-12D/B Paveway II LGB [Mk82]</a></b> -
				 Guided Weapon<br/>
				Surface Max: 7.4 km. Land Max: 7.4 km. </td></tr><tr><td> <a class="preview pull-right" href="images/DB3000/weapon_1814.jpg" target="_blank"><i class="fa fa-picture-o"></i></a><b><a href="weapon/1814/">Mk82 500lb LDGP</a></b> -
				(1954) Bomb<br/>
				Surface Max: 1.9 km. Land Max: 1.9 km. </td></tr> </tbody>
</table>

In [16]:
for table_number in range(number_of_tables):
    _index = table_header_dict['index'][table_number]
    _header = table_header_dict['header'][table_number][0]

    if _header == "General data:":      
        df_table_1 = read_table_one(soup, table_number)
        logging.debug(f'Table 1 returned successfully.')
    elif _header == "Sensors / EW:":
        df_table_2 = read_table_two(soup, table_number)
        logging.debug(f'Table 2 returned successfully.')

#     _aircraft_data = combine_tables_into_dataframe(df_table_1, df_table_2)
#     logging.debug(f'Table 1 and 2 combined successfully.')

DEBUG:root:Table 1 returned successfully.
DEBUG:root:Length of flat_list is: 3.
DEBUG:root:flat_list is: ['Star Safire III [FLIR] - (Group, AN/AAQ-21/22) Infrared\n\t\t\t\t\t\tInfrared, Surveillance Camera Max Range: 55.6 km', 'Star Safire III [CCD] - (Group, AN/AAQ-21/22) Visual\n\t\t\t\t\t\tLLTV, Surveillance Camera Max Range: 55.6 km', 'Star Safire III [Laser Designator] - (Group, AN/AAQ-21/22) Laser Designator\n\t\t\t\t\t\tLaser Target Designator & Ranger (LTD/R) Max Range: 27.8 km'].
DEBUG:root:Table 2 returned successfully.


### Investigation

In [14]:
table = soup.find_all('table')
table

[<table class="table table-striped table-hover">
 <thead>
 <tr>
 <th><b><u>General data:</u></b></th>
 <th></th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td>Type: Recon</td>
 <td>Crew: 2</td>
 </tr>
 <tr>
 <td>Min Speed: 230 kt</td>
 <td>Max Speed: 460 kt</td>
 </tr>
 <tr>
 <td>Wingspan: 12.0 m</td>
 <td>Height: 3.7 m</td>
 </tr>
 <tr>
 <td>Length: 14.7 m</td>
 <td>Max Payload: 0 kg</td>
 </tr>
 <tr>
 <td>Empty Weight: 4590 kg</td>
 <td>Max Weight: 8235 kg</td>
 </tr>
 <tr>
 <td>Operator: Air Force</td>
 <td>Commissioned: 1980</td>
 </tr>
 <tr>
 <td style="max-width: 100px;">Propulsion: 2x TFE731-2-2B</td>
 <td>
 </td>
 </tr>
 </tbody>
 </table>]

In [15]:
len(table)

1

In [16]:
print(type(table))
print(type(table[0]))

<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>


In [17]:
_ = table[0].find_all('th')
type(_)

bs4.element.ResultSet

In [18]:
_

[<th><b><u>General data:</u></b></th>, <th></th>]

In [19]:
_[0].find_all('u')

[<u>General data:</u>]

In [20]:
header = [i.text for i in _]
header

['General data:', '']

#### How code is laid out in main:

In [33]:
number_of_tables, table_header_dict = count_tables_on_page(soup)

DEBUG:root:There are 3 tables on this page.
DEBUG:root:Index: 0
DEBUG:root:Header: ['General data:', '']
DEBUG:root:Index: 1
DEBUG:root:Header: ['Sensors / EW:']
DEBUG:root:Index: 2
DEBUG:root:Header: ['Weapons / Loadouts:']


In [35]:
for i in range(number_of_tables):
    _index = table_header_dict['index'][i]
    _header = table_header_dict['header'][i][0]
    
    if _header == "General data:":
        df_table_1 = read_table_one(soup, table_number = i)
        logging.debug(f'Table 1 returned successfully.')
    elif _header == "Sensors / EW:":
        df_table_2 = read_table_two(soup, table_number = i)
        logging.debug(f'Table 2 returned successfully.')
        

DEBUG:root:Table 1 returned successfully.
DEBUG:root:Length of flat_list is: 1.
DEBUG:root:flat_list is: ['China Type 222 - (J-7) Radar\n\t\t\t\t\tRadar, Air Search, 3D Short-Range Max Range: 18.5 km'].
DEBUG:root:Table 2 returned successfully.


In [ ]:
df_table_1 = read_table_one(soup)
logging.debug(f'Table 1 returned successfully.')
df_table_2 = read_table_two(soup)
logging.debug(f'Table 2 returned successfully.')

DEBUG:root:Table 1 returned successfully.


IndexError: list index out of range

### Solution

In [47]:
table = soup.find_all('table')
len(table)

3

In [53]:
index_list = []
header_list = []

In [54]:
for i in range(len(table)):
    header = table[i].find_all('th')
    header_string = [i.text for i in header]
    index_list.append(i)
    header_list.append(header_string)
    logging.info(f"Index: {i}")
    logging.info(f"Header: {header_string}")

INFO:root:Index: 0
INFO:root:Header: ['General data:', '']
INFO:root:Index: 1
INFO:root:Header: ['Sensors / EW:']
INFO:root:Index: 2
INFO:root:Header: ['Weapons / Loadouts:']


In [55]:
_ = {"index": index_list,
    "header": header_list}

In [ ]:
def count_tables_on_page(soup):
    table = soup.find_all('table')
    logging.debug(f"There are {len(table)} tables on this page.")

    if len(table) > 0:
        index_list, header_list = _get_table_header(table)
    else:
        index_list = []
        header_list = []
        logging.error(f"No tables found.")
    
    table_header_dict = {"index": index_list,
            "header": header_list}
    
    return table_header_dict

In [ ]:
def _get_table_header(table):
    for i in range(len(table)):
        header = table[i].find_all('th')
        header_string = [i.text for i in header]
        index_list.append(i)
        header_list.append(header_string)
        logging.debug(f"Index: {i}")
        logging.debug(f"Header: {header_string}")
    
    return index_list, header_list

In [58]:
_['index'][0]

0

In [63]:
_['header'][1][0]

'Sensors / EW:'